In [7]:
import json, re
from tqdm import tqdm
from collections import Counter, defaultdict
from datetime import datetime
from random import sample
import matplotlib.pyplot as plt
import numpy as np
from glob import glob

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

from gensim.models import Word2Vec

In [32]:
embeddings = {}

with open('./embeddings/glove.6B.50d.txt') as f:
    lines = f.read().strip().split('\n')
    for line in lines:
        word = line.split()[0]
        vector = np.array(line.split()[1:], dtype=np.float)
        embeddings[word] = vector

In [33]:
embeddings[',']

array([ 0.013441,  0.23682 , -0.16899 ,  0.40951 ,  0.63812 ,  0.47709 ,
       -0.42852 , -0.55641 , -0.364   , -0.23938 ,  0.13001 , -0.063734,
       -0.39575 , -0.48162 ,  0.23291 ,  0.090201, -0.13324 ,  0.078639,
       -0.41634 , -0.15428 ,  0.10068 ,  0.48891 ,  0.31226 , -0.1252  ,
       -0.037512, -1.5179  ,  0.12612 , -0.02442 , -0.042961, -0.28351 ,
        3.5416  , -0.11956 , -0.014533, -0.1499  ,  0.21864 , -0.33412 ,
       -0.13872 ,  0.31806 ,  0.70358 ,  0.44858 , -0.080262,  0.63003 ,
        0.32111 , -0.46765 ,  0.22786 ,  0.36034 , -0.37818 , -0.56657 ,
        0.044691,  0.30392 ])

In [34]:
title = 'Donald Trump beat Hillary Clinton in the 2016 United States Presidential Elections'.lower()

In [35]:
title_embed = np.array([embeddings[word] for word in title.split() if word in embeddings])

In [40]:
avg_title_embed = np.mean(title_embed, axis=0)

In [42]:
len(avg_title_embed)

50

In [2]:
pattern = re.compile('[\W_]+')

In [3]:
s = "hellow hillary's the one person i don't talk about"

In [4]:
pattern.sub(' ', s).lower().split()

['hellow',
 'hillary',
 's',
 'the',
 'one',
 'person',
 'i',
 'don',
 't',
 'talk',
 'about']

In [5]:
subs = '''Conservative Liberal Republican democrats hillaryclinton The_Donald news worldnews politics'''.split()
print(subs)

['Conservative', 'Liberal', 'Republican', 'democrats', 'hillaryclinton', 'The_Donald', 'news', 'worldnews', 'politics']


In [14]:
filelist =  glob('./data/RC_right_sample')#[i for i in glob('./data/*') if i.split('_')[-1] in subs]
filelist

['./data/RC_right_sample']

In [15]:
data = defaultdict(list)
failed = []

for file in filelist:
    with open(file) as f:
        print(file)
        for i, line in enumerate(tqdm(f)):
#             if i > 100000: break
            post = json.loads(line)
            try:
#                 data[post['subreddit']].append(pattern.sub(' ', post['body']).lower().split()) #, post['author']))
                data['left'].append(pattern.sub(' ', post['body']).lower().split()) #, post['author']))
            except Exception as e:
                failed.append((i, e))

for i in data:
    print(i, '\t', len(data[i]))

5269it [00:00, 52685.28it/s]

./data/RC_right_sample


3000000it [01:24, 35630.63it/s]

left 	 3000000


In [16]:
politics =  Word2Vec(data['left'])

In [17]:
print(politics.wv.vectors.shape)

(77762, 100)


In [20]:
politics.wv.most_similar(['hillary'], topn=100)

[('hilary', 0.9165266156196594),
 ('hrc', 0.8640804290771484),
 ('killary', 0.8613182306289673),
 ('shillary', 0.8323041200637817),
 ('hiliary', 0.7956461310386658),
 ('hitlery', 0.7916281223297119),
 ('hilliary', 0.7711724638938904),
 ('trump', 0.7584517598152161),
 ('illary', 0.7565233707427979),
 ('crooked', 0.7488961219787598),
 ('clinton', 0.7429749965667725),
 ('she', 0.7193477749824524),
 ('bernie', 0.7032478451728821),
 ('hilldog', 0.6914997100830078),
 ('hilllary', 0.6804174184799194),
 ('obama', 0.6775376200675964),
 ('hillery', 0.6491295099258423),
 ('hitlary', 0.6467891335487366),
 ('sanders', 0.6378533840179443),
 ('killery', 0.6359661221504211),
 ('hildabeast', 0.6315698027610779),
 ('hilldawg', 0.626826286315918),
 ('her', 0.6259810924530029),
 ('klanton', 0.6104066967964172),
 ('mrs', 0.5947917103767395),
 ('kaine', 0.5880374312400818),
 ('djt', 0.5752742290496826),
 ('chelsea', 0.5716038942337036),
 ('hellary', 0.568051278591156),
 ('he', 0.5668107271194458),
 ('mclose

In [21]:
politics.save('./embeddings/RC_right.model')

In [2]:
models = {}

In [3]:
files = glob('./embeddings/*.model')

In [8]:
test = Word2Vec.load('embeddings/RC_news.model')

ValueError: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 192 from PyObject

In [13]:
for i in files:
    models[i.split('/')[-1]] = Word2Vec.load(i)

In [14]:
models.keys()

dict_keys(['RC_news.model', 'RC_left.model', 'RC_right.model', 'RC_politics.model', 'RC_worldnews.model'])

In [50]:
target = 'trump'
mdl = 'RC_right.model'

print(target)

output = []

for i in models[mdl].wv.most_similar(target, topn=20):
    output.append(i[0])

print(', '.join(output))
#     print(i[0],)#, '\t', i[1])

trump
djt, hillary, shillary, hilary, killary, bernie, dt, him, he, hrc, pence, tump, hitlery, romney, obama, sanders, clinton, gop, hilldog, hiliary
